# Import Libraries

In [16]:
import sqlite3
import pandas as pd
import os

# Import Googel GenAI

In [17]:
from google import genai
from google.colab import userdata

genai_client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

# Create Database Schema (in SQLITE)

### Load sql files to create **company.db** and populate it with dummy data

In [18]:
if os.path.exists('company.db'):
    os.remove('company.db')
    print("company.db exists and has been deleted.")
conn = sqlite3.connect('company.db')
cursor = conn.cursor()

sql_files = ['departments.sql', 'designations.sql', 'employees.sql']

for sql_file in sql_files:
    with open(sql_file, 'r') as f:
        sql_script = f.read()
    cursor.executescript(sql_script)

conn.commit()

print("Database schema created successfully.")

company.db exists and has been deleted.
Database schema created successfully.


In [19]:
response = conn.execute('Select * from employees limit 10')

for row in response:
    print(row)


(1, '24-4825933', 'Gamaliel', 'Sharply', '9/13/1996', 'Male', 8, 5, 107661.1, '1/31/2014', 'gsharply0@ustream.tv')
(2, '95-4173137', 'Gabriella', 'Watters', '11/5/1983', 'Female', 10, 4, 73694.62, '10/7/2014', 'gwatters1@about.com')
(3, '82-7771215', 'Inessa', 'Walak', '8/30/1966', 'Female', 6, 3, 115560.19, '2/28/2017', 'iwalak2@jimdo.com')
(4, '18-8172302', 'Bax', 'Yearne', '12/25/1998', 'Male', 4, 4, 56820.36, '7/19/2019', 'byearne3@wix.com')
(5, '23-9199907', 'Lind', 'Catenot', '6/22/2002', 'Male', 3, 8, 31527.96, '7/29/2006', 'lcatenot4@foxnews.com')
(6, '70-4150638', 'Jameson', 'Westwater', '11/14/1969', 'Male', 1, 12, 133973.27, '5/2/2011', 'jwestwater5@house.gov')
(7, '27-6994952', 'Bord', 'Lightfoot', '10/2/1965', 'Male', 1, 12, 32941.34, '4/6/2015', 'blightfoot6@istockphoto.com')
(8, '87-8468483', 'Nickie', 'Ingreda', '11/28/1980', 'Male', 3, 5, 73938.17, '12/10/2000', 'ningreda7@parallels.com')
(9, '99-5785435', 'Tiphani', 'Halloran', '2/22/1976', 'Female', 6, 16, 96778.55, 

# Prompt

In [57]:
prompt = '''
## 📌 Final Structured Prompt

### ROLE
You are an **expert SQL query generator** specializing in **SQLite**. You deeply understand **relational database design, query optimization, and handling foreign key relationships**. Your job is to transform **natural language questions** into **optimized, error-free SQLite queries** that correctly use the **employees, designations, and departments tables**.

---

### CONTEXT
- The database is an **Employees Management Database** consisting of at least three tables:
  1. **employees** – stores employee details such as id, name, designation_id, department_id, joining_date, salary, etc.
  2. **designations** – stores job roles such as id, designation_name, level, etc.
  3. **departments** – stores department details such as id, department_name, location, etc.

- The schema is linked by **foreign keys**:
  - `employees.designation_id → designations.id`
  - `employees.department_id → departments.id`

- The user provides **plain English questions** about employees, their designations, and departments.
- The AI must generate **SQLite-compatible queries** that are **optimized and executable**.

---

### DATABASE SCHEMA

create table employees (
	id INT,
	employee_code VARCHAR(50),
	first_name VARCHAR(50),
	last_name VARCHAR(50),
	date_of_birth DATE,
	gender VARCHAR(50),
	designation_id INT,
	department_id INT,
	salary DECIMAL(8,2),
	hire_date DATE,
	email VARCHAR(50)
);

create table departments (
	id INT,
	department_name VARCHAR(50),
	department_code VARCHAR(9),
	department_location VARCHAR(50),
	department_phone VARCHAR(50),
	department_email VARCHAR(50)
);

create table designations (
	id INT,
	designation_name VARCHAR(50)
);

---

### TASK
1. Read the user’s natural language request carefully.
2. Identify which **tables, fields, and relationships** are needed.
3. Construct a **valid SQLite query** that answers the question.
4. Always use **clear JOIN conditions** to handle foreign keys properly.
5. Optimize queries for SQLite performance (e.g., avoid unsupported features like `RIGHT JOIN` or `FULL OUTER JOIN`).
6. If assumptions are required (e.g., column names), include them as **SQL comments** above the query.
7. Ensure the final query is **error-free and executable in SQLite**.

---

### CONSTRAINTS
- ✅ Always output the query inside **SQL code fences**.
- ✅ Use **table aliases** for readability (e.g., `e`, `d`, `ds`).
- ✅ Use **aggregation (SUM, COUNT, AVG), GROUP BY, HAVING, subqueries, and window functions** when needed.
- ❌ Do not explain queries outside of SQL comments.
- ❌ Do not use syntax unsupported by SQLite.
- ❌ Do not guess random column names—stick to logical assumptions with comments.

---

### **EXAMPLES**

**Example 1: Simple Lookup**

  * **User Query:** "List all employees who work in the HR department"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT e.id, e.name FROM employees e INNER JOIN departments d ON e.department_id = d.id WHERE d.department_name = 'HR';"
    }
    ```

---

**Example 2: Complex Join and Aggregation**

  * **User Query:** "Who are the top 5 highest-paid employees along with their designation and department?"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT e.name, e.salary, ds.designation_name, d.department_name FROM employees e INNER JOIN designations ds ON e.designation_id = ds.id INNER JOIN departments d ON e.department_id = d.id ORDER BY e.salary DESC LIMIT 5;"
    }
    ```

---

**Example 3: Ambiguous Query**

  * **User Query:** "Show me recently joined employees"
  * **Expected Output:**
    ```json
    {
      "status": "clarification_needed",
      "response": "Could you please define what 'recently' means? For example, 'in the last 30 days', 'this year', or 'since January 2025'."
    }
    ```

---

**Example 4: Impossible Query**

  * **User Query:** "Which employee has the company car assigned?"
  * **Expected Output:**
    ```json
    {
      "status": "error",
      "response": "I cannot answer this question as the database does not contain information about company cars."
    }
    ```

---

### **OUTPUT FORMAT**

Your final response must be a single JSON object with two keys:

1.  `"status"`: A string with one of three possible values: `"success"`, `"clarification_needed"`, or `"error"`.
2.  `"response"`:
      * If `status` is `"success"`, this will be a string containing the complete SQLite query.
      * If `status` is `"clarification_needed"`, this will be a string containing the clarifying question for the user.
      * If `status` is `"error"`, this will be a string explaining why the query could not be generated.

'''


# Text-to-SQL Generator

In [58]:
import json
def get_sql_query(genai_client, prompt, user_query):

  # https://www.geeksforgeeks.org/python/formatted-string-literals-f-strings-python/
  contents = f"""
  {prompt}

  Here's the user query in english you need to work on:
  {user_query}
  """
  response = genai_client.models.generate_content(model='gemini-2.5-flash', contents=contents)
  # print(response)

  # Access the usage_metadata attribute
  usage_metadata = response.usage_metadata

  # Print the different token counts
  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count: {response.usage_metadata.thoughts_token_count}")
  print(f"Output Token Count: {usage_metadata.candidates_token_count}")
  print(f"Total Token Count: {usage_metadata.total_token_count}")

  output = json.loads(response.text.replace('```json', '').replace('```', ''))

  return output


### SQL Executer

In [59]:
import sqlite3
import pandas as pd

def execute_query(query, db_name='company.db'):

    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Execute the query
        print(f"\nExecuting query on '{db_name}':\n{query}")
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Get column names from the cursor description
        columns = [description[0] for description in cursor.description]

        # Format results as a dataframe for easier use
        results_as_dict = [dict(zip(columns, row)) for row in results]
        results_df = pd.DataFrame(results_as_dict)

        print("Query executed successfully.")
        return results_df

    except sqlite3.Error as e:
        print(f"Database error executing query: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    finally:
        if conn:
            conn.close()

### Text2SQL

In [60]:
def text2sql(genai_client, prompt, user_query):
  output = get_sql_query(genai_client, prompt, user_query)
  if output['status'] == 'success':
    results = execute_query(output['response'])
    return results
  return output

# Testing

In [61]:
# Example 1:
text2sql(genai_client, prompt, "Show me list of departments")

Input Token Count: 1311
Thoughts Token Count: 33
Output Token Count: 29
Total Token Count: 1373

Executing query on 'company.db':
SELECT department_name FROM departments;
Query executed successfully.


,department_name
0,Legal
1,Sales
2,Support
3,Product Management
4,Legal
5,Research and Development
6,Services
7,Sales
8,Accounting
9,Product Management


In [62]:
# Example 2: List employees with their designation and department
print("\n--- Example 2: Employees with Designation and Department ---")
text2sql(genai_client, prompt, "List all employees with their designation and department")



--- Example 2: Employees with Designation and Department ---
Input Token Count: 1314
Thoughts Token Count: 296
Output Token Count: 83
Total Token Count: 1693

Executing query on 'company.db':
SELECT e.first_name, e.last_name, ds.designation_name, d.department_name FROM employees AS e INNER JOIN designations AS ds ON e.designation_id = ds.id INNER JOIN departments AS d ON e.department_id = d.id;
Query executed successfully.


,first_name,last_name,designation_name,department_name
0,Gamaliel,Sharply,Programmer Analyst I,Legal
1,Gabriella,Watters,Sales Representative,Product Management
2,Inessa,Walak,Engineer IV,Support
3,Bax,Yearne,Software Engineer III,Product Management
4,Lind,Catenot,Web Designer III,Sales
...,...,...,...,...
995,Tynan,Yerrell,Administrative Assistant I,Support
996,Joann,Pitrelli,Administrative Assistant I,Product Management
997,Fee,Scaddon,Programmer Analyst I,Product Management
998,Sutherland,Stubbe,Cost Accountant,Services


In [63]:

# Example 3: Count of employees by department
print("\n--- Example 3: Employee Count by Department ---")
text2sql(genai_client, prompt, "How many employees are in each department?")



--- Example 3: Employee Count by Department ---
Input Token Count: 1314
Thoughts Token Count: 117
Output Token Count: 64
Total Token Count: 1495

Executing query on 'company.db':
SELECT d.department_name, COUNT(e.id) AS employee_count FROM employees e INNER JOIN departments d ON e.department_id = d.id GROUP BY d.department_name;
Query executed successfully.


,department_name,employee_count
0,Accounting,102
1,Business Development,110
2,Engineering,49
3,Legal,229
4,Product Management,90
5,Research and Development,81
6,Sales,150
7,Services,52
8,Support,88
9,Training,49


In [64]:

# Example 4: Employees hired after a specific date
print("\n--- Example 4: Employees Hired After 2015 ---")
text2sql(genai_client, prompt, "Show me employees hired after January 1st, 2015")



--- Example 4: Employees Hired After 2015 ---
Input Token Count: 1321
Thoughts Token Count: 140
Output Token Count: 55
Total Token Count: 1516

Executing query on 'company.db':
SELECT id, first_name, last_name, hire_date FROM employees WHERE hire_date > '2015-01-01';
Query executed successfully.


,id,first_name,last_name,hire_date
0,4,Bax,Yearne,7/19/2019
1,5,Lind,Catenot,7/29/2006
2,6,Jameson,Westwater,5/2/2011
3,7,Bord,Lightfoot,4/6/2015
4,9,Tiphani,Halloran,4/14/2005
...,...,...,...,...
581,996,Tynan,Yerrell,6/7/2020
582,997,Joann,Pitrelli,9/10/2000
583,998,Fee,Scaddon,8/31/2016
584,999,Sutherland,Stubbe,3/17/2006


In [65]:

# Example 5: Average salary by designation
print("\n--- Example 5: Average Salary by Designation ---")
text2sql(genai_client, prompt, "What is the average salary for each designation?")



--- Example 5: Average Salary by Designation ---
Input Token Count: 1315
Thoughts Token Count: 128
Output Token Count: 67
Total Token Count: 1510

Executing query on 'company.db':
SELECT ds.designation_name, AVG(e.salary) AS average_salary FROM employees e INNER JOIN designations ds ON e.designation_id = ds.id GROUP BY ds.designation_name;
Query executed successfully.


,designation_name,average_salary
0,Administrative Assistant I,91121.579304
1,Community Outreach Specialist,88977.106129
2,Cost Accountant,92591.698980
3,Engineer IV,87750.637865
4,Graphic Designer,89242.428654
5,Programmer Analyst I,96331.437647
6,Sales Representative,90526.057304
7,Software Engineer III,87830.465825
8,Statistician III,92090.982727
9,Web Designer III,88712.653118


In [68]:

# Example 6: List all designations
print("\n--- Example 6: List all Designations ---")
text2sql(genai_client, prompt, "List all job titles")



--- Example 6: List all Designations ---
Input Token Count: 1310
Thoughts Token Count: 83
Output Token Count: 30
Total Token Count: 1423

Executing query on 'company.db':
SELECT DISTINCT designation_name FROM designations;
Query executed successfully.


,designation_name
0,Cost Accountant
1,Community Outreach Specialist
2,Web Designer III
3,Software Engineer III
4,Graphic Designer
5,Engineer IV
6,Statistician III
7,Programmer Analyst I
8,Administrative Assistant I
9,Sales Representative


In [67]:

# Example 7: List all departments
print("\n--- Example 7: List all Departments ---")
text2sql(genai_client, prompt, "List all available departments")


--- Example 7: List all Departments ---
Input Token Count: 1310
Thoughts Token Count: 51
Output Token Count: 33
Total Token Count: 1394

Executing query on 'company.db':
SELECT d.department_name FROM departments AS d;
Query executed successfully.


,department_name
0,Legal
1,Sales
2,Support
3,Product Management
4,Legal
5,Research and Development
6,Services
7,Sales
8,Accounting
9,Product Management


In [69]:
# Example 8: A Complex Query
print("\n--- Example 8: A Complex Query ---")
text2sql(genai_client, prompt, "Show employee with highest salary in his department.")


--- Example 8: A Complex Query ---
Input Token Count: 1315
Thoughts Token Count: 1068
Output Token Count: 110
Total Token Count: 2493

Executing query on 'company.db':
SELECT e.first_name, e.last_name, e.salary, d.department_name FROM (SELECT id, first_name, last_name, department_id, salary, RANK() OVER (PARTITION BY department_id ORDER BY salary DESC) AS rnk FROM employees) AS e INNER JOIN departments AS d ON e.department_id = d.id WHERE e.rnk = 1;
Query executed successfully.


,first_name,last_name,salary,department_name
0,Bradley,Waudby,145460.33,Legal
1,Helsa,Hoofe,149605.47,Sales
2,Abigale,MacEllen,149818.95,Support
3,Hoebart,Noseworthy,145230.62,Product Management
4,Caty,Groves,147643.91,Legal
5,Marcie,Brunicke,149078.63,Research and Development
6,Marin,Salatino,149391.14,Services
7,Rocky,Maslin,143774.30,Sales
8,Denis,Brandes,147517.23,Accounting
9,Page,Mapstone,147426.64,Product Management


In [70]:
# Example 9: Non-employee database related query
print("\n--- Example 8: Non-employee database related query ---")
text2sql(genai_client, prompt, "List highest ordered product")


--- Example 8: Non-employee database related query ---
Input Token Count: 1310
Thoughts Token Count: 257
Output Token Count: 50
Total Token Count: 1617


{'status': 'error',
 'response': 'I cannot answer this question as the database does not contain information about products or orders. The available tables are employees, departments, and designations.'}

In [72]:
# Example 10: Another Complex Query (Department with highest average salary)
print("\n--- Example 10: Department with Highest Average Salary ---")
text2sql(genai_client, prompt, "Which department has the highest average salary?")


--- Example 10: Department with Highest Average Salary ---
Input Token Count: 1314
Thoughts Token Count: 171
Output Token Count: 73
Total Token Count: 1558

Executing query on 'company.db':
SELECT d.department_name, AVG(e.salary) AS average_salary FROM employees e INNER JOIN departments d ON e.department_id = d.id GROUP BY d.department_name ORDER BY average_salary DESC LIMIT 1;
Query executed successfully.


,department_name,average_salary
0,Research and Development,98972.428395


In [73]:
# Example 11: A More Complex Query
print("\n--- Example 11: A More Complex Query ---")
text2sql(genai_client, prompt, "Show employee, designation wise with highest salary in his department.")


--- Example 11: A More Complex Query ---
Input Token Count: 1318
Thoughts Token Count: 1502
Output Token Count: 207
Total Token Count: 3027

Executing query on 'company.db':
WITH RankedEmployees AS (
    SELECT
        e.first_name,
        e.last_name,
        e.salary,
        ds.designation_name,
        d.department_name,
        ROW_NUMBER() OVER (PARTITION BY e.designation_id, e.department_id ORDER BY e.salary DESC) as rn
    FROM
        employees e
    INNER JOIN
        designations ds ON e.designation_id = ds.id
    INNER JOIN
        departments d ON e.department_id = d.id
)
SELECT
    first_name,
    last_name,
    salary,
    designation_name,
    department_name
FROM
    RankedEmployees
WHERE
    rn = 1;
Query executed successfully.


,first_name,last_name,salary,designation_name,department_name
0,Alexei,Callicott,128651.63,Cost Accountant,Legal
1,Velvet,Bielfeld,146507.49,Cost Accountant,Sales
2,Chrotoem,Ickowics,119713.90,Cost Accountant,Support
3,Anatol,Cleever,105274.34,Cost Accountant,Product Management
4,Consolata,De la Yglesia,139858.22,Cost Accountant,Legal
...,...,...,...,...,...
193,Mariejeanne,Crowdy,145401.68,Sales Representative,Legal
194,Dorotea,Cussons,138030.28,Sales Representative,Research and Development
195,Dermot,Slemming,102966.67,Sales Representative,Business Development
196,Port,Coxall,81584.34,Sales Representative,Legal
